# 批量改變圖片輸入大小(若只train一張圖可不做)

In [18]:
from PIL import Image
import os.path
import glob
def convertjpg(jpgfile,outdir,width=512,height=512):
    img=Image.open(jpgfile)
    try:
        new_img=img.resize((width,height),Image.BILINEAR)   
        new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))
    except Exception as e:
        print(e)
for jpgfile in glob.glob("data/*.jpg"):        #原圖存放的資料夾(data資料夾jpg形式的圖片)
    convertjpg(jpgfile,"data_resize")          #resize後要放的資料夾


# 讀入一張照片(RGB)並轉成黑白

In [19]:
import keras
import tensorflow as tf
from skimage.io import imread, imsave
from skimage.color import rgb2gray, gray2rgb, rgb2lab, lab2rgb
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
import numpy as np


def image_color_test():
    # this method to let's know something about RGB and LAB color space
    test_image_gray = rgb2gray(imread('data_resize/wei.jpg'))
    test_image_rgb = imread('data_resize/wei.jpg')

    print(test_image_gray.shape)
    print(test_image_gray)
    print(test_image_rgb.shape)


if __name__ == '__main__':
    image_color_test()

(512, 512)
[[0.84586941 0.84586941 0.84586941 ... 0.86237412 0.86237412 0.86237412]
 [0.84586941 0.84586941 0.84586941 ... 0.86237412 0.86237412 0.86237412]
 [0.84586941 0.84586941 0.84586941 ... 0.86237412 0.86237412 0.86237412]
 ...
 [0.58682588 0.5832098  0.60733529 ... 0.57238784 0.56901686 0.54016706]
 [0.57923569 0.57196588 0.58684157 ... 0.57267059 0.56901686 0.54715412]
 [0.59771255 0.59127608 0.59438706 ... 0.56649412 0.56312314 0.56284039]]
(512, 512, 3)


# 準備訓練data
* 將rgb轉成lab
* 把lab的L作為輸入，AB作為label

In [20]:
import keras
import tensorflow as tf
from skimage.io import imread, imsave
from skimage.color import rgb2gray, gray2rgb, rgb2lab, lab2rgb
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os


def get_train_data(img_file):
    image = img_to_array(load_img(img_file))
    image_shape = image.shape
    image = np.array(image, dtype=float)

    x = rgb2lab(1.0 / 255 * image)[:, :, 0]          #把RGB形式的image轉成LAB形式(X是輸入(亮度L))
    y = rgb2lab(1.0 / 255 * image)[:, :, 1:]         #把RGB形式的image轉成LAB形式(Y是LABEL(AB))
    y=y/128
    x = x.reshape(1, image_shape[0], image_shape[1], 1)
    y = y.reshape(1, image_shape[0], image_shape[1], 2)
    return x, y, image_shape

# 建構模型&設定參數

In [21]:
def build_model():
    model = Sequential()
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
    model.compile(optimizer='adam', loss='mse')
    return model


def train():
    x, y, img_shape = get_train_data('data_resize/wei.jpg')
    model = build_model()

    num_epochs = 6000
    batch_size = 6
    model_file = 'simple_model.h5'

    model.fit(x, y, batch_size=1, epochs=1000)
    model.save(model_file)

# 執行訓練

In [22]:
train()

Epoch 1/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.1247
Epoch 2/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.1293
Epoch 3/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.0918
Epoch 4/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0459
Epoch 5/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.0236
Epoch 6/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.0253
Epoch 7/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.0198
Epoch 8/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.0133
Epoch 9/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.0123
Epoch 10/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0118
Epoch 11/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0109
Epoch 12/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.0094
Epoch 13/1000
1/1 [=====================

# 建立讀取權重並執行上色預測的function

In [25]:
def colorize():
    x, y, image_shape = get_train_data('data_resize/wei.jpg')
    model = build_model()
    model.load_weights('simple_model.h5')
    output = model.predict(x)
    output *= 128
    tmp = np.zeros((512, 512, 3))
    tmp[:, :, 0] = x[0][:, :, 0]
    tmp[:, :, 1:] = output[0]
    imsave("test_image_result.png", lab2rgb(tmp))
    imsave("test_image_gray.png", rgb2gray(lab2rgb(tmp)))

# 執行上色預測

In [26]:
colorize()